## Loading Data

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers
import tensorflow_addons as tfa
from tensorflow import data as tf_data

In [ ]:
train_60,val_20,test_20=tfds.load("oxford_iiit_pet",split=["train+test[:20%]","test[20%:60%]","test[60%:]"],as_supervised=True)

resizing=layers.Resizing(224,224)
train_60=train_60.map(lambda x,y:(resizing(x),y))
val_20=val_20.map(lambda x,y:(resizing(x),y))
test_20=test_20.map(lambda x,y:(resizing(x),y))

batch_size = 64

train_60 = train_60.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
val_20 = val_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test_20 = test_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()



In [ ]:
train,test=tfds.load("oxford_iiit_pet",split=["train","test"],as_supervised=True)

resizing=layers.Resizing(224,224)
train=train.map(lambda x,y:(resizing(x),y))
test=test.map(lambda x,y:(resizing(x),y))

batch_size = 64

train = train.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test = test.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [ ]:
F10_train=tfds.load("oxford_iiit_pet",split=[f"train[:{r}%]+train[{r+10}%:]" for r in range(0,100,10)],as_supervised=True)
F10_val=tfds.load("oxford_iiit_pet",split=[f"train[{r}%:{r+10}%]" for r in range(0,100,10)],as_supervised=True)


## Model

In [ ]:
base_model=keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3),pooling="avg")

In [ ]:
base_model.trainable=False

In [ ]:
inputs=keras.Input(shape=(224,224,3))   
preprocessing=tf.keras.applications.resnet.preprocess_input(inputs)
x=base_model(preprocessing, training=False)
x=layers.Dense(256,activation="relu")(x)
outputs=layers.Dense(37)(x)

model=keras.Model(inputs,outputs)

loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizers=keras.optimizers.Adam(learning_rate=0.001)

F1 = tfa.metrics.FBetaScore(num_classes=37)
model.compile(optimizer=optimizers,loss=loss_fn,metrics=['accuracy'])

In [ ]:
model.summary(show_trainable=True) 

In [ ]:
model.fit(train_60,epochs=10,validation_data=val_20)